In [179]:
import numpy as np
import tensorflow as tf
import pandas as pd
from ipywidgets import widgets
from sklearn import preprocessing
from tensorflow import keras 
from tensorflow.keras import layers, regularizers,Model, utils
%matplotlib inline 
import matplotlib.pyplot as plt
import sklearn
from tensorflow.keras.callbacks import TensorBoard
from datetime import time

In [25]:
dataPath_input = widgets.Text(description="Path to CSV:", value="outputs/")
dataPath_input

Text(value='outputs/', description='Path to CSV:')

In [80]:
dataByLocation = pd.read_csv(dataPath_input.value)
data = dataByLocation.drop(columns=["Time", "Longitude", "Latitude"])

In [81]:
data = dataByLocation.drop(columns=["Time", "Longitude", "Latitude"])
data

,Speed,LatAcceleration,LongAcceleration,SteerTorque,SteerRate,SteerAngle,FLWheelSpeed,FRWheelSpeed,RRWheelSpeed,RLWheelSpeed
0,75.155687,-501.740000,0.013309,0.0,0.000000,1.700000,75.159684,75.113947,74.932737,74.926684
1,75.260991,-501.740000,0.000000,0.0,-0.967652,1.800000,75.006872,75.095745,75.014983,75.029818
2,75.227563,-501.740000,0.000000,0.0,-3.648693,1.810812,75.262530,75.182388,74.810142,74.737943
3,75.467328,-501.720768,0.112396,0.0,0.000000,2.100000,75.521644,75.467706,75.129178,74.946233
4,75.500000,-501.740000,0.103073,0.0,0.000000,2.100000,75.536009,75.604389,75.237599,75.336790
...,...,...,...,...,...,...,...,...,...,...
37853,0.000000,-501.740000,0.000000,0.0,0.000000,-229.100000,0.000000,0.000000,0.000000,0.000000
37854,0.000000,-501.733100,0.000000,0.0,0.000000,-229.100000,0.000000,0.000000,0.000000,0.000000
37855,0.000000,-501.740000,0.000000,0.0,0.000000,-229.100000,0.000000,0.000000,0.000000,0.000000
37856,0.000000,-501.740000,0.000000,0.0,0.000000,-229.100000,0.000000,0.000000,0.000000,0.000000


## Normalize Data

In [83]:
data = data.values[:]
min_max_scaler = preprocessing.MinMaxScaler()
data_normalized = min_max_scaler.fit_transform(data)
data_normalized = pd.DataFrame(data_normalized)

## Window Data

In [87]:
features = ['Speed',
            'LatAcceleration',
            'LongAcceleration',
            'SteerTorque',
            'SteerRate',
            'SteerAngle',
            'FLWheelSpeed',
            'FRWheelSpeed',
            'RRWheelSpeed',
            'RLWheelSpeed']
data_df = data_normalized

In [89]:
sequenceLength = 5
stackedData, stackedLabels = np.array([]), np.array([])

stackedData = []

# split can_data into subsampled sequences
for i in range(len(data_df)-sequenceLength):
    stackedData.append(data_df[i:i+sequenceLength])
    
stackedData = np.array(stackedData)

## Generate Labels

In [101]:
data_normalized.shape

(37857, 10)

In [104]:
longitudeLabels = []
latitudeLabels = []

for i,window in enumerate(stackedData[:-1]):
    last = dataByLocation.iloc[i]
    lastLong = last.Longitude
    lastLat = last.Latitude
    cur = dataByLocation.iloc[i+5]
    curLong = cur.Longitude
    curLat = cur.Latitude
    
    longitudeLabels.append(curLong - lastLong)
    latitudeLabels.append(curLat - lastLat)

longitudeLabels = np.array(longitudeLabels)
latitudeLabels = np.array(latitudeLabels)

pd.DataFrame({"longDeltas": longitudeLabels, "latDeltas": latitudeLabels}).describe()

,longDeltas,latDeltas
count,37851.000000,37851.000000
mean,0.000005,-0.000032
std,0.000043,0.000044
min,-0.000122,-0.000130
25%,0.000000,-0.000080
50%,0.000000,0.000000
75%,0.000000,0.000000
max,0.000175,0.000084


## Build Neural Network

### MLP

In [107]:
trainingData = stackedData[:-1]
trainingData.shape

(37851, 5, 10)

In [108]:
#specify input dimensionality
numberOfSamples = trainingData.shape[0]
inputDimension = trainingData[0].shape[0]*trainingData[0].shape[1]
outputDimension = 1

#construct our neural network
hiddenLayerSizes = [inputDimension, 32, 16, 8, 4, 2]

#set up our input layer
inputLayer = layers.Input(shape=(inputDimension, ))

#set up our hidden layers
curLayer = 0
previousLayer = inputLayer
for curLayerSize in hiddenLayerSizes:
    previousLayer = layers.Dense(curLayerSize,
                                 activation='sigmoid', 
                                 name=str(curLayerSize)+'_hiddenLayer',
                                 kernel_regularizer=regularizers.L2(0.001)
                                )(previousLayer)
    
outputLayer = layers.Dense(outputDimension, activation='sigmoid')(previousLayer)

#compile our model
ourModel = Model(inputs=inputLayer, outputs=[outputLayer], name='longitude_mlp')
ourModel.compile(loss='mean_squared_error', metrics=['acc'], optimizer='adam')
    

In [109]:
#train the mlp
#initially we have(N, 28, 28); we want (N, 28*28)
trainingDataFlat = trainingData.reshape((-1, inputDimension))
ourModel.fit(trainingDataFlat, longitudeLabels, epochs=10)

Epoch 1/10
1183/1183 [==============================] - 2s 1ms/step - loss: 0.3070 - acc: 0.3259
Epoch 2/10
1183/1183 [==============================] - 2s 1ms/step - loss: 0.0596 - acc: 0.6790
Epoch 3/10
1183/1183 [==============================] - 1s 1ms/step - loss: 0.0114 - acc: 0.6729
Epoch 4/10
1183/1183 [==============================] - 1s 1ms/step - loss: 0.0040 - acc: 0.6768
Epoch 5/10
1183/1183 [==============================] - 1s 1ms/step - loss: 0.0015 - acc: 0.6760
Epoch 6/10
1183/1183 [==============================] - 1s 1ms/step - loss: 6.3401e-04 - acc: 0.6799
Epoch 7/10
1183/1183 [==============================] - 1s 1ms/step - loss: 2.8226e-04 - acc: 0.6800
Epoch 8/10
1183/1183 [==============================] - 1s 1ms/step - loss: 1.3071e-04 - acc: 0.6795
Epoch 9/10
1183/1183 [==============================] - 1s 1ms/step - loss: 6.2510e-05 - acc: 0.6781
Epoch 10/10
1183/1183 [==============================] - 1s 1ms/step - loss: 3.0677e-05 - acc: 0.6818


### CNN

In [111]:
trainingData.shape

(37851, 5, 10)

In [132]:
#specify input dimensionality
numberOfSamples = trainingData.shape[0]
inputDimension = trainingData[0].shape[0]*trainingData[0].shape[1]
outputDimension = 1

#construct our neural network
hiddenLayerSizes = [inputDimension, 32, 16, 8, 4, 2]

#set up our input layer
inputLayer = layers.Input(shape=(inputDimension, 1))

#set up our hidden layers
curLayer = 0
previousLayer = inputLayer
for curLayerSize in hiddenLayerSizes:
    previousLayer = layers.Conv1D(curLayerSize, 5,
                                  activation='sigmoid',
                                  name=str(curLayerSize)+'_hiddenLayer',
                                  kernel_regularizer=regularizers.L2(0.001),
                                  input_shape=(numberOfSamples, 5, 10)
                                 )(previousLayer)
    
outputLayer = layers.Dense(outputDimension, activation='sigmoid')(previousLayer)

#compile our model
ourModel = Model(inputs=inputLayer, outputs=[outputLayer], name='longitude_cnn')
ourModel.compile(loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer='adam')
    

In [133]:
#train the mlp
trainingDataFlat = trainingData.reshape((-1, inputDimension))
ourModel.fit(trainingDataFlat, longitudeLabels, epochs=10)

Epoch 1/10
1183/1183 [==============================] - 3s 3ms/step - loss: 0.1368 - root_mean_squared_error: 0.3425
Epoch 2/10
1183/1183 [==============================] - 3s 2ms/step - loss: 0.0131 - root_mean_squared_error: 0.1069
Epoch 3/10
1183/1183 [==============================] - 3s 3ms/step - loss: 0.0024 - root_mean_squared_error: 0.0426
Epoch 4/10
1183/1183 [==============================] - 3s 3ms/step - loss: 0.0010 - root_mean_squared_error: 0.0275
Epoch 5/10
1183/1183 [==============================] - 3s 2ms/step - loss: 4.5273e-04 - root_mean_squared_error: 0.0191
Epoch 6/10
1183/1183 [==============================] - 3s 3ms/step - loss: 2.0555e-04 - root_mean_squared_error: 0.0134
Epoch 7/10
1183/1183 [==============================] - 3s 2ms/step - loss: 9.5787e-05 - root_mean_squared_error: 0.0094
Epoch 8/10
1183/1183 [==============================] - 3s 3ms/step - loss: 4.6903e-05 - root_mean_squared_error: 0.0067
Epoch 9/10
1183/1183 [==========================

### Resnet

In [196]:
#specify input dimensionality
numberOfSamples = trainingData.shape[0]
inputDimension = trainingData[0].shape[0]*trainingData[0].shape[1]
outputDimension = 1

#construct our neural network
hiddenLayerSizes = [inputDimension, 32, 32, 16, 16, 8, 4, 2]

#set up our input layer
inputLayer = layers.Input(shape=(inputDimension, 1))

#set up our hidden layers
curLayer = 0
previousPreviousLayer = None
previousLayer = inputLayer
for i,curLayerSize in enumerate(hiddenLayerSizes):
    previousPreviousLayer = previousLayer
    previousLayer = layers.Conv1D(curLayerSize, 5,
                                  activation='sigmoid',
                                  name='hiddenLayer_'+str(i),
                                  kernel_regularizer=regularizers.L2(0.001),
                                  input_shape=(numberOfSamples, 5, 10)
                                 )(previousLayer * previousPreviousLayer)
outputLayer = layers.Dense(outputDimension, activation='linear')(previousLayer)

#compile our model
ourModel = Model(inputs=inputLayer, outputs=[outputLayer], name='longitude_resnet')
ourModel.compile(loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer='adam')
    

In [197]:
#train the mlp
dataFlat = trainingData.reshape((-1, inputDimension))

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(dataFlat, longitudeLabels, test_size=0.5, shuffle=False)

tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))

history = ourModel.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), callbacks=[tensorboard])

Epoch 1/10
592/592 [==============================] - 4s 5ms/step - loss: 0.0605 - root_mean_squared_error: 0.1098 - val_loss: 0.0047 - val_root_mean_squared_error: 0.0024
Epoch 2/10
592/592 [==============================] - 3s 4ms/step - loss: 0.0037 - root_mean_squared_error: 0.0021 - val_loss: 0.0018 - val_root_mean_squared_error: 0.0015
Epoch 3/10
592/592 [==============================] - 3s 4ms/step - loss: 0.0016 - root_mean_squared_error: 0.0013 - val_loss: 9.8404e-04 - val_root_mean_squared_error: 0.0011
Epoch 4/10
592/592 [==============================] - 3s 5ms/step - loss: 8.5719e-04 - root_mean_squared_error: 9.5660e-04 - val_loss: 5.3705e-04 - val_root_mean_squared_error: 7.5179e-04
Epoch 5/10
592/592 [==============================] - 3s 4ms/step - loss: 4.5721e-04 - root_mean_squared_error: 6.7209e-04 - val_loss: 2.5588e-04 - val_root_mean_squared_error: 5.0275e-04
Epoch 6/10
592/592 [==============================] - 3s 4ms/step - loss: 2.0825e-04 - root_mean_squared

In [208]:
# make a prediction
ynew = ourModel.predict(X_train)

ynew.shape
# show the inputs and predicted outputs
# for i in range(len(X_val)):
# 	print("Predicted=%s, actual=%s" % (ynew[i], y_val[i]))

(18925, 18, 1)

In [200]:
X_train.shape

(18925, 50)

In [201]:
y_train.shape

(18925,)